In [ ]:
from google.colab import drive, files

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/'My Drive'/440-Adv-ML-Project/CSDS440-f22-P2/code/models
%pwd

/content/drive/My Drive/440-Adv-ML-Project/CSDS440-f22-P2/code/models


'/content/drive/My Drive/440-Adv-ML-Project/CSDS440-f22-P2/code/models'

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms

from resnet import Resnet18

In [ ]:
# get model
normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465],
    std=[0.2023, 0.1994, 0.2010],
)

# define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize,
])

dataset = datasets.CIFAR10(
    root='./data', train=False,
    download=True, transform=transform,
)

Files already downloaded and verified


In [ ]:
model = Resnet18(10)
# model = Resnet20()
# model = Resnet34()
model.load_state_dict(torch.load('./saved_models/Resnet18_CIFAR10.pth'))

<All keys matched successfully>

In [ ]:
! pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 6.7 MB/s 


In [ ]:
# set the model to inference mode -- IMPORTANT!!
model.eval()

# dummy input tensor
dummy_input = torch.randn(1, *[3,224,224], requires_grad=True)

# Export the model
torch.onnx.export(model, dummy_input, './saved_models/Resnet18_CIFAR10.onnx', export_params=True, opset_version=10, do_constant_folding=True)

print(" ")
print('Model has been converted to ONNX')

 
Model has been converted to ONNX


In [ ]:
%ls saved_models/

Resnet18_CIFAR10.onnx  Resnet18_CIFAR10.pth


In [ ]:
!pip install git+https://github.com/onnx/onnx-tensorflow.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-u5b6_3u1
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-u5b6_3u1
     |████████████████████████████████| 1.1 MB 6.8 MB/s 
  Created wheel for onnx-tf: filename=onnx_tf-1.10.0-py3-none-any.whl size=226663 sha256=288720702366707be7662665c8eb2a9fb7eacecc8bdc40bf94680a1ec5e14a61
  Stored in directory: /tmp/pip-ephem-wheel-cache-uszjs2ft/wheels/e9/b7/67/8abcf50ca14ef4375302d30abf9f4242e06be09b829f5fd307
Successfully built onnx-tf


In [ ]:
import onnx
from tensorflow.python.tools.import_pb_to_tensorboard import import_to_tensorboard
from onnx_tf.backend import prepare
onnx_model = onnx.load("./saved_models/Resnet18_CIFAR10.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("./saved_models/Resnet18_CIFAR10.pb")

In [ ]:
%ls ./saved_models

Resnet18_CIFAR10.onnx  Resnet18_CIFAR10.pb/  Resnet18_CIFAR10.pth
